In [ ]:
import os
import requests
import json
import re
from typing import List, Dict
from bs4 import BeautifulSoup
from openai import OpenAI
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

In [ ]:
MODEL = "llama3.2"
openai = OpenAI(base_url = "http://localhost:11434/v1", api_key = "ollama")

In [ ]:
class HotelListing:
    def __init__(self, name, price, url, features = None):
        self.name = name
        self.price = price
        self.url = url
        self.features = features or []
    def to_dict(self):
        return {
            "name": self.name,
            "price": self.price,
            "url": self.url,
            "features": self.features
        }

In [ ]:
class BookingParser:
    def __init__(self, url, headers = None):
        self.url = url
        self.headers = headers or {"User-Agent": "Mozilla/5.0"}
        self.listings = []
        self.fetch_and_parse()

    def fetch_and_parse(self):
        try:
            request = requests.get(self.url, headers = self.headers, timeout = 10)
            request.raise_for_status()
        except Exception as e:
            print(f"Page download error: {e}")
            return

        soup = BeautifulSoup(request.content, "html.parser")

        hotel_cards = soup.find_all("div", {"data-stid": "property-listing-results"})

        if not hotel_cards:
            hotel_cards = soup.find_all("div", class_ = re.compile("property-listing|property-card-card-results"))

        for card in hotel_cards[:10]:
            listing = self._parse_hotel_card(card)
            if listing:
                self.listings.append(listing)

    def _parse_hotel_card(self, card):
        try:
            name_element = card.find("a", {"data-stid": "open-hotel-information"})
            if not name_element:
                name_element = card.find("h3") or car.find("span", class_ = re.compile("is-visually-hidden"))
            name = name_element.get_text(strip = True) if name_element else "name unknown"

            price_element = card.find("span", {"class": "uitk-badge-base-text"})

            price_text = price_element.get_text(strip = True) if price_element else "0"
            price_match = request.search(r'(\d+)', price_text.replace('$', ''))
            price = int(price_match.group(1)) if price_match else 0

            link_element = card.find("a", href = True)
            url = "https://www.hotels.com" + link_element["href"] if link_element else ""

            features = []
            feature_spans = card.select('[data-stid="sp-content-list"]')
            if feature_spans:
                items = feature_spans[0].select('li[data-stid^="sp-content-item"]')
                
                for item in items:
                    text = item.get_text(strip=True)
                    if text:
                        features.append(text.lower())

            card_text = card.get_text().lower()
            if "wi-fi" in card_text or "wifi" in card_text:
                features.append("wifi")
                if "breakfest" in card_text:
                    features.append("breakfest")

            return HotelListing(name, price, url, features)
        except Exception as e:
            print(f"Parsing hotel card error: {e}")
            return None

    def get_listings(self):
        return [listing.to_dict() for listing in self.listings]
        

In [ ]:
def make_prompt(listings: List[Dict], user_preferences: Dict):
    prompt = (
        "You are an assistant and help a user in accommodation choosing.\n"
        "Below is a list of hotel offers and user preferences.\n"
        "HOTELS OFERTS:\n"
        f"{json.dumps(listings, ensure_ascii = False, indent = 1)}\n\n"
        "USER PREFERENCES:\n"
        f"{json.dumps(user_preferences, ensure_ascii = False, indent = 1)}\n\n"
        "For every ofert:\n"
        "1) Assess suitability in 0-10 rate (where 10 = ideal suitability)\n"
        "2) Give 2-3 short reasons for your assessment\n"
        "3) Please indicate if the price is within your budget\n"
        "Finally, list the TOP 3 best offers with justification.\n"
    )
    return prompt

In [ ]:
def analyze_listings(listings: List[Dict], preferences: Dict):
    if not listings:
        print("No offers to analyze.")
        return None

    prompt = make_prompt(listings, preferences)

    try:
        response = openai.chat.completions.create(
            model = MODEL,
            messages = [
                {
                    "role": "system",
                    "content": "You are an expert in choosing the best accommodation.\n" 
                               "You analyze offers and advise users."
                },
                {"role": "user", "content": prompt}
            ]
        )

        result = response.choices[0].message.content
        return result

    except Exception as e:
        print(f"Communication error with LLM: {e}")
        return None

In [ ]:
def main():
    url = ("https://www.hotels.com/Hotel-Search?destination=Warsaw%20-%20Eastern%20Poland%2C%20Poland&d1=2025-10-18&startDate=2025-10-18&d2=2025-10-20&endDate=2025-10-20&adults=1&rooms=1&regionId=6057142&sort=RECOMMENDED&theme=&userIntent=&semdtl=&categorySearch=&useRewards=false&children=&latLong&pwaDialog=&daysInFuture&stayLength")

    preferences = {
        "max_price": 200,
        "must_have": ["wifi", "breakfest"],
        "number_of_rooms": 1,
        "localization": "Warsaw"
    }

    print("🔍 Oferts downloading from Hotels.com..")
    parser = BookingParser(url)
    listings = parser.get_listings()

    print(f"✅ Found {len(listings)} offerts\n")
    print("="*60)

    print("FOUND OFFERTS:\n")
    for i, listing in enumerate(listings, 1):
        print(f"\n{i}. {listing['name']}")
        print(f"Amount: {listing['price']} pln")
        print(f"Features: {', '.join(listing['features']) if listing['features'] else 'Informations lack.'}")

        analysis = analyze_listings(listings, preferences)

        if analysis:
            print(analysis)
        else:
            print("❌ Analysis failed")

if __name__ == "__main__":
    main()
        